## Purpose of this notebook
Merge the SERVICES and CPTEVENTS table, which contain the services and procedures for which the patients were billed, respectively.

In [73]:
import pandas as pd
# load the services table
services_df = pd.read_csv('../../data/raw/SERVICES.csv')
services_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73343 entries, 0 to 73342
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ROW_ID        73343 non-null  int64 
 1   SUBJECT_ID    73343 non-null  int64 
 2   HADM_ID       73343 non-null  int64 
 3   TRANSFERTIME  73343 non-null  object
 4   PREV_SERVICE  14668 non-null  object
 5   CURR_SERVICE  73343 non-null  object
dtypes: int64(3), object(3)
memory usage: 3.4+ MB


In [74]:
# filter the table with qualifying hospital admission ids
hadmid = pd.read_csv('../../data/intermediate/inter022620a.csv')
services_df = services_df.merge(hadmid['HADM_ID'], left_on='HADM_ID', right_on='HADM_ID', how='right')
service_count = services_df.CURR_SERVICE.value_counts()
service_count

MED      24276
CMED      8643
CSURG     4482
SURG      4410
NSURG     3226
TRAUM     3127
NMED      2905
OMED      2159
VSURG      944
TSURG      690
ORTHO      684
NBB        243
GU         174
GYN        172
ENT        127
PSURG      121
OBS        112
DENT         5
PSYCH        1
Name: CURR_SERVICE, dtype: int64

In [48]:
# top 12 account for the majority of services
# Note NBB (newborn baby) services should not be here, need to filter out very young patients
# replace remaining services with MED
services_df = services_df[services_df.CURR_SERVICE!='NBB']

top11 = service_count.index[0:11].tolist()

services_df.loc[~services_df['CURR_SERVICE'].isin(top11),'CURR_SERVICE'] = 'MED'
service_count = services_df.CURR_SERVICE.value_counts()
service_count

MED      25007
CMED      8643
CSURG     4482
SURG      4410
NSURG     3226
TRAUM     3127
NMED      2905
OMED      2159
VSURG      944
TSURG      690
ORTHO      684
Name: CURR_SERVICE, dtype: int64

In [49]:
# Since the number of services rows is greater than the number of admissions, it follows that some admissions 
# correspond to multiple services. Crosstabulate hospital admission ids and services
crosstab_serv = pd.crosstab(services_df['HADM_ID'], services_df['CURR_SERVICE'])
print("The number of qulifying admisions with services is ", len(crosstab_serv))

The number of qulifying admisions with services is  43164


In [50]:
crosstab_serv.columns

Index(['CMED', 'CSURG', 'MED', 'NMED', 'NSURG', 'OMED', 'ORTHO', 'SURG',
       'TRAUM', 'TSURG', 'VSURG'],
      dtype='object', name='CURR_SERVICE')

In [51]:
# It would be better to keep track of which categorical variables belong together
serv_new_colnames = ['CURR_SERVICE_' + column for column in crosstab_serv.columns]
serv_new_colnames

['CURR_SERVICE_CMED',
 'CURR_SERVICE_CSURG',
 'CURR_SERVICE_MED',
 'CURR_SERVICE_NMED',
 'CURR_SERVICE_NSURG',
 'CURR_SERVICE_OMED',
 'CURR_SERVICE_ORTHO',
 'CURR_SERVICE_SURG',
 'CURR_SERVICE_TRAUM',
 'CURR_SERVICE_TSURG',
 'CURR_SERVICE_VSURG']

In [52]:
# now rename the columns
crosstab_serv.columns = serv_new_colnames
crosstab_serv.columns

Index(['CURR_SERVICE_CMED', 'CURR_SERVICE_CSURG', 'CURR_SERVICE_MED',
       'CURR_SERVICE_NMED', 'CURR_SERVICE_NSURG', 'CURR_SERVICE_OMED',
       'CURR_SERVICE_ORTHO', 'CURR_SERVICE_SURG', 'CURR_SERVICE_TRAUM',
       'CURR_SERVICE_TSURG', 'CURR_SERVICE_VSURG'],
      dtype='object')

In [53]:
# as an additional feature calculate the total number of services performed during each admission
crosstab_serv['CURR_SERVICE_TOTAL'] = crosstab_serv.sum(axis=1)

In [54]:
# load the procedures column
cpt_df = pd.read_csv('../../data/raw/CPTEVENTS.csv', low_memory=False)
cpt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573146 entries, 0 to 573145
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ROW_ID            573146 non-null  int64  
 1   SUBJECT_ID        573146 non-null  int64  
 2   HADM_ID           573146 non-null  int64  
 3   COSTCENTER        573146 non-null  object 
 4   CHARTDATE         101545 non-null  object 
 5   CPT_CD            573146 non-null  object 
 6   CPT_NUMBER        573128 non-null  float64
 7   CPT_SUFFIX        22 non-null      object 
 8   TICKET_ID_SEQ     471601 non-null  float64
 9   SECTIONHEADER     573125 non-null  object 
 10  SUBSECTIONHEADER  573125 non-null  object 
 11  DESCRIPTION       101545 non-null  object 
dtypes: float64(2), int64(3), object(7)
memory usage: 52.5+ MB


In [55]:
# the column we need is subsectionheader (procedure category)
cpt_df = cpt_df.merge(hadmid['HADM_ID'], left_on='HADM_ID', right_on='HADM_ID', how='right')
cpt_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 531065 entries, 0 to 531064
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ROW_ID            525870 non-null  float64
 1   SUBJECT_ID        525870 non-null  float64
 2   HADM_ID           531065 non-null  int64  
 3   COSTCENTER        525870 non-null  object 
 4   CHARTDATE         90465 non-null   object 
 5   CPT_CD            525870 non-null  object 
 6   CPT_NUMBER        525855 non-null  float64
 7   CPT_SUFFIX        5 non-null       object 
 8   TICKET_ID_SEQ     435405 non-null  float64
 9   SECTIONHEADER     525854 non-null  object 
 10  SUBSECTIONHEADER  525854 non-null  object 
 11  DESCRIPTION       90465 non-null   object 
dtypes: float64(4), int64(1), object(7)
memory usage: 52.7+ MB


In [56]:
# since number of HADM_ID values > SUBSECTIONHEADER values, it follows some admission do not have procedures performed
cpt_df = cpt_df[~cpt_df.SUBSECTIONHEADER.isna()]
len(cpt_df)

525854

In [57]:
# Let's see the unique value counts
cpt_df.SUBSECTIONHEADER.value_counts()

Hospital inpatient services                                                                                                                                                  255150
Critical care services                                                                                                                                                       100292
Pulmonary                                                                                                                                                                     90480
Consultations                                                                                                                                                                 23968
Cardiovascular system                                                                                                                                                         14733
Dialysis                                                                                            

In [58]:
len(cpt_df.SUBSECTIONHEADER.value_counts())

53

In [59]:
# There are 53 procedures, however procedures under 200 count are a minority, merge with the top serveice
proc_count = cpt_df.SUBSECTIONHEADER.value_counts()
minority_proc = proc_count[proc_count<200].index
minority_proc

Index(['Hospital observation services', 'Case management services',
       'Other procedures',
       'Hydration, therapeutic, prophylactic, diagnostic injections and infusions, and chemotherapy and other highly complex drug or highly complex biologic agent administration',
       'Confirmatory consultation for a new or established patient (deleted codes)',
       'Female genital system', 'Moderate (conscious) sedation',
       'Bone/joint studies', 'Mediastinum and diaphragm',
       'Hematology and coagulation', 'Office/other outpatient services',
       'Conscious sedation (deleted codes)', 'Maternity care and delivery',
       'Non-face-to-face nonphysician services', 'Ophthalmology',
       'Central nervous system assessments/tests (neuro-cognitive, mental status, speech testing)',
       'Eye and ocular adnexa', 'Male genital system',
       'Team Conferences (deleted codes)', 'Radiologic guidance',
       'End-Stage Renal Disease Services (deleted codes)',
       'Preventive med

In [60]:
cpt_df.loc[cpt_df.SUBSECTIONHEADER.isin(minority_proc), 'SUBSECTIONHEADER'] = 'Hospital inpatient services'
cpt_df.SUBSECTIONHEADER.value_counts()

Hospital inpatient services                          256435
Critical care services                               100292
Pulmonary                                             90480
Consultations                                         23968
Cardiovascular system                                 14733
Dialysis                                               8966
Respiratory system                                     8800
Musculoskeletal system                                 5362
Digestive system                                       3819
Nervous system                                         2683
Diagnostic ultrasound                                  1943
Follow-up inpatient consultations (deleted codes)      1933
Integumentary system                                   1504
Prolonged services                                     1041
Cardiovascular                                          763
Diagnostic imaging                                      648
Physical medicine and rehabilitation    

In [61]:
# crosstabulate and rename the columns
crosstab_cpt = pd.crosstab(cpt_df['HADM_ID'], cpt_df['SUBSECTIONHEADER'])
crosstab_cpt.columns = ['SUBSECTIONHEADER_' + column for column in crosstab_cpt.columns]
crosstab_cpt.columns

Index(['SUBSECTIONHEADER_Cardiovascular',
       'SUBSECTIONHEADER_Cardiovascular system',
       'SUBSECTIONHEADER_Consultations',
       'SUBSECTIONHEADER_Critical care services',
       'SUBSECTIONHEADER_Diagnostic imaging',
       'SUBSECTIONHEADER_Diagnostic ultrasound', 'SUBSECTIONHEADER_Dialysis',
       'SUBSECTIONHEADER_Digestive system',
       'SUBSECTIONHEADER_Follow-up inpatient consultations (deleted codes)',
       'SUBSECTIONHEADER_Hemic and lymphatic systems',
       'SUBSECTIONHEADER_Hospital inpatient services',
       'SUBSECTIONHEADER_Integumentary system',
       'SUBSECTIONHEADER_Musculoskeletal system',
       'SUBSECTIONHEADER_Nervous system',
       'SUBSECTIONHEADER_Noninvasive vascular diagnostic studies',
       'SUBSECTIONHEADER_Operating microscope (deleted code)',
       'SUBSECTIONHEADER_Physical medicine and rehabilitation',
       'SUBSECTIONHEADER_Prolonged services', 'SUBSECTIONHEADER_Psychiatry',
       'SUBSECTIONHEADER_Pulmonary', 'SUBSECTIONHEAD

In [62]:
# add a feature with the total number of columns
crosstab_cpt['SUBSECTIONHEADER_TOTAL'] = crosstab_cpt.sum(axis=1)

In [63]:
# let's merge crosstabulated services and procedures data frames
crosstab_serv = crosstab_serv.merge(crosstab_cpt, left_index=True, right_index=True)
len(crosstab_serv)

38211

In [64]:
#check for missing values
import numpy as np
# check if there are missing values
np.isnan(crosstab_serv).sum()

CURR_SERVICE_CMED                                                     0
CURR_SERVICE_CSURG                                                    0
CURR_SERVICE_MED                                                      0
CURR_SERVICE_NMED                                                     0
CURR_SERVICE_NSURG                                                    0
CURR_SERVICE_OMED                                                     0
CURR_SERVICE_ORTHO                                                    0
CURR_SERVICE_SURG                                                     0
CURR_SERVICE_TRAUM                                                    0
CURR_SERVICE_TSURG                                                    0
CURR_SERVICE_VSURG                                                    0
CURR_SERVICE_TOTAL                                                    0
SUBSECTIONHEADER_Cardiovascular                                       0
SUBSECTIONHEADER_Cardiovascular system                          

In [65]:
# let's merge admissions table
crosstab_serv.reset_index(inplace=True)
hadmid = hadmid.merge(crosstab_serv, left_on='HADM_ID', right_on='HADM_ID', how='left')
np.isnan(hadmid).sum()

SUBJECT_ID                                                      0
HADM_ID                                                         0
HOSPITAL_EXPIRE_FLAG                                            0
OUTPUT_LABEL                                                    0
1Y_PRIOR_ADM                                                    0
                                                             ... 
SUBSECTIONHEADER_Pulmonary                                   5196
SUBSECTIONHEADER_Respiratory system                          5196
SUBSECTIONHEADER_Special services, procedures and reports    5196
SUBSECTIONHEADER_Urinary system                              5196
SUBSECTIONHEADER_TOTAL                                       5196
Length: 61, dtype: int64

In [66]:
# 5196 admissions are missing info on procedures and services
# check to see what fraction is in the positive label, otherwize if the majority is in the negative label 
# we can eliminate those samples, since we have an excess of negative label
hadmid.drop('SUBJECT_ID', axis=1, inplace=True)
hadmid[hadmid.SUBSECTIONHEADER_TOTAL.isna()].groupby('OUTPUT_LABEL')['HADM_ID'].count()

OUTPUT_LABEL
0.0    5005
1.0     191
Name: HADM_ID, dtype: int64

In [67]:
# The majority is in the negative class, so we can delete those samples
hadmid=hadmid[~hadmid.SUBSECTIONHEADER_TOTAL.isna()]
len(hadmid)

38211

In [68]:
# final check for missing values
np.isnan(hadmid).sum().sum()

0

In [71]:
# inspect the columns
hadmid.columns.tolist()

['HADM_ID',
 'HOSPITAL_EXPIRE_FLAG',
 'OUTPUT_LABEL',
 '1Y_PRIOR_ADM',
 'LENGTH_OF_STAY_DAYS',
 'AGE',
 'ADMISSION_LOCATION_EMERGENCY ROOM ADMIT',
 'ADMISSION_LOCATION_PHYS REFERRAL/NORMAL DELI',
 'ADMISSION_LOCATION_TRANSFER FROM HOSP/EXTRAM',
 'DISCHARGE_LOCATION_DISC-TRAN CANCER/CHLDRN H',
 'DISCHARGE_LOCATION_DISCH-TRAN TO PSYCH HOSP',
 'DISCHARGE_LOCATION_HOME',
 'DISCHARGE_LOCATION_HOME HEALTH CARE',
 'DISCHARGE_LOCATION_HOSPICE-HOME',
 'DISCHARGE_LOCATION_LEFT AGAINST MEDICAL ADVI',
 'DISCHARGE_LOCATION_LONG TERM CARE HOSPITAL',
 'DISCHARGE_LOCATION_REHAB/DISTINCT PART HOSP',
 'DISCHARGE_LOCATION_SHORT TERM HOSPITAL',
 'DISCHARGE_LOCATION_SNF',
 'ETHNICITY_BLACK',
 'ETHNICITY_HISPANIC',
 'ETHNICITY_OTHER',
 'ETHNICITY_WHITE',
 'GENDER_M',
 'CURR_SERVICE_CMED',
 'CURR_SERVICE_CSURG',
 'CURR_SERVICE_MED',
 'CURR_SERVICE_NMED',
 'CURR_SERVICE_NSURG',
 'CURR_SERVICE_OMED',
 'CURR_SERVICE_ORTHO',
 'CURR_SERVICE_SURG',
 'CURR_SERVICE_TRAUM',
 'CURR_SERVICE_TSURG',
 'CURR_SERVICE_VSURG

In [72]:
# No missing values, save the intermediate dataset
hadmid.to_csv('../../data/intermediate/inter022620b.csv')